In [2]:
# check python version & nessessary package
!python --version

!pip show transformers sentencepiece pytorch_lightning editdistance scikit-learn numpy tqdm | python -c "import sys, re; print(''.join(line for line in sys.stdin if re.search(r'Name|Version', line)), end='\t')"


Python 3.8.18
Name: transformers
Version: 4.18.0
Name: sentencepiece
Version: 0.1.96
Name: pytorch-lightning
Version: 0.8.1
Name: editdistance
Version: 0.6.0
Name: scikit-learn
Version: 0.24.2
Name: numpy
Version: 1.22.3
Name: tqdm
Version: 4.64.0
	


In [4]:
import os

# path to folder 'code' in BGCA repo
os.chdir('../code')

In [5]:
os.getcwd()

'c:\\Workspace\\Tutorial\\BGCA\\code'

In [7]:
python main.py --task aste --name me5-base  --seed 42  --dataset cross_domain  --model_name_or_path intfloat/multilingual-e5-base  --paradigm extraction-universal  --n_gpu 0  --train_batch_size 16  --gradient_accumulation_steps 2  --eval_batch_size 16  --learning_rate 3e-4  --num_train_epochs 25  --save_last_k 3  --n_runs 1  --clear_model  --save_best  --data_gene  --data_gene_extract  --data_gene_extract_epochs 25  --data_gene_epochs 25  --init_tag english  --do_train  --do_eval  --use_same_model  --data_gene_wt_constrained  --model_filter  --train_by_pair 

python main.py --task aste --name me5-base  --seed 42  --dataset cross_domain  --model_name_or_path intfloat/multilingual-e5-base  --paradigm extraction-universal  --n_gpu 1  --train_batch_size 16  --gradient_accumulation_steps 2  --eval_batch_size 16  --learning_rate 3e-4  --num_train_epochs 5  --save_last_k 3  --n_runs 1  --clear_model  --save_best  --data_gene  --data_gene_extract  --data_gene_extract_epochs 5  --data_gene_epochs 5  --init_tag english  --do_train  --do_eval  --use_same_model  --data_gene_wt_constrained  --model_filter  --train_by_pair 

usage: main.py [-h] --task TASK [--device DEVICE] --dataset DATASET
               [--data_dir DATA_DIR] [--model_name_or_path MODEL_NAME_OR_PATH]
               --paradigm PARADIGM [--do_train] [--do_eval] [--name NAME]
               [--commit COMMIT] [--save_last_k SAVE_LAST_K] [--n_runs N_RUNS]
               [--clear_model] [--save_best] [--nrows NROWS] [--train_by_pair]
               [--target_domain TARGET_DOMAIN] [--no_greedy] [--beam BEAM]
               [--max_seq_length MAX_SEQ_LENGTH] [--n_gpu N_GPU]
               [--train_batch_size TRAIN_BATCH_SIZE]
               [--eval_batch_size EVAL_BATCH_SIZE]
               [--gradient_accumulation_steps GRADIENT_ACCUMULATION_STEPS]
               [--learning_rate LEARNING_RATE]
               [--num_train_epochs NUM_TRAIN_EPOCHS] [--seed SEED]
               [--weight_decay WEIGHT_DECAY] [--adam_epsilon ADAM_EPSILON]
               [--warmup_steps WARMUP_STEPS] [--init_tag INIT_TAG]
               [--data_gene] [--data_gene_epoc

In [9]:
import logging
import re
import os
os.chdir('../code/')
import random

import torch
from torch.utils.data import DataLoader
# from tqdm import tqdm, trange
from tqdm.notebook import tqdm, trange
from transformers import (AdamW, AutoModelForSeq2SeqLM, AutoModel,
                          AutoTokenizer, get_linear_schedule_with_warmup)
from constants import *
from data_utils import (ABSADataset, filter_none, filter_invalid,
                        get_dataset, get_inputs, normalize_augment)
from model_utils import (prepare_constrained_tokens, prepare_tag_tokens)
from main import *
from data_utils import *

In [10]:
# Load arguments from the JSON file

import json
import argparse

arg_path = '../outputs/aste/cross_domain/me5-base/seed-42/rest14-laptop14/args.json'
with open(arg_path, 'r') as file:
    args_dict = json.load(file)

# Create a namespace from the dictionary
args = argparse.Namespace(**args_dict)

args.model_name_or_path = '../outputs/aste/cross_domain/me5-base/seed-42/rest14-laptop14/extract_aste-model'
print(args)


Namespace(adam_epsilon=1e-08, beam=1, clear_model=True, commit=None, data_dir='../outputs/aste/cross_domain\\me5-base/seed-42/rest14-laptop14\\data', data_gene=True, data_gene_aug_num=None, data_gene_aug_ratio=None, data_gene_decode=None, data_gene_epochs=25, data_gene_extract=True, data_gene_extract_epochs=25, data_gene_extract_none_remove_ratio=0, data_gene_min_length=0, data_gene_none_remove_ratio=0, data_gene_none_word_num=1, data_gene_num_beam=1, data_gene_same_model=False, data_gene_top_p=0.9, data_gene_wt_constrained=True, dataset='cross_domain', device='cuda', do_eval=True, do_train=True, eval_batch_size=16, extract_model=None, gene_model=None, gradient_accumulation_steps=2, inference_dir='../outputs/aste/cross_domain\\me5-base/seed-42/rest14-laptop14\\inference', init_tag='english', learning_rate=0.0003, max_seq_length=128, model_filter=True, model_filter_skip_none=False, model_name_or_path='../outputs/aste/cross_domain/me5-base/seed-42/rest14-laptop14/extract_aste-model', n_g

In [11]:
model = AutoModel.from_pretrained(args.model_name_or_path).to(args.device)
tokenizer = AutoTokenizer.from_pretrained(args.model_name_or_path, use_fast=False)

In [15]:
input_list = [
    "เป็นร้านที่ไปยากนิดนึง หากไม่คุ้นเคยเส้นทาง เห็นมีหลายคนหลงทาง เป็นร้านอาหารสำหรับนักท่องเที่ยว หรือเอาแขกต่างชาติมา พื้นที่ร้านมหึมาและดูอลังการ เน้นบรรยากาศและการตกแต่งเป็นส่วนใหญ่ ส่วนมากจะเป็นของที่สะสม ส่วนอาหารถือว่าพอได้ แต่ยังดูธรรมดาหากเปรียบเทียบกับร้านอาหารไทยแนวเดียวกัน รวมถึงราคาก็จะสูงหน่อย"
]

target_list = [""]

In [16]:
dataset = ABSADataset(args, tokenizer, inputs=input_list, targets=target_list)


c:\Workspace\Tutorial\BGCA\.conda\lib\site-packages\transformers\tokenization_utils_base.py:2263: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [17]:
def infer_new(args, dataset, model, tokenizer, name=None, is_constrained=False, constrained_vocab=None, keep_mask=False, **decode_dict):
    dataloader = DataLoader(dataset, batch_size=args.eval_batch_size, num_workers=4)

    if keep_mask:
        # can't skip special directly, will lose extra_id
        unwanted_tokens = [tokenizer.eos_token, tokenizer.unk_token, tokenizer.pad_token]
        unwanted_ids = tokenizer.convert_tokens_to_ids(unwanted_tokens)
        def filter_decode(ids):
            ids = [i for i in ids if i not in unwanted_ids]
            tokens = tokenizer.convert_ids_to_tokens(ids)
            sentence = tokenizer.convert_tokens_to_string(tokens)
            return sentence

    # inference
    inputs, outputs, targets = [], [], []
    
    model.eval()
    with torch.no_grad():
        for batch in tqdm(dataloader, desc='Generating'):
            if is_constrained:
                prefix_fn_obj = Prefix_fn_cls(tokenizer, constrained_vocab, batch['source_ids'].to(args.device))  # need fix
                prefix_fn = lambda batch_id, sent: prefix_fn_obj.get(batch_id, sent)
            else:
                prefix_fn = None

            outs_dict = model.generate(input_ids=batch['source_ids'].to(args.device),
                                        attention_mask=batch['source_mask'].to(args.device),
                                        # max_length=128,
                                        # prefix_allowed_tokens_fn=prefix_fn,
                                        # output_scores=True,
                                        # return_dict_in_generate=True,
                                        # **decode_dict,
                                        )
            outs = outs_dict["sequences"]

            if keep_mask:
                input_ = [filter_decode(ids) for ids in batch["source_ids"]]
                dec = [filter_decode(ids) for ids in outs]
                target = [filter_decode(ids) for ids in batch["target_ids"]]
            else:
                input_ = [tokenizer.decode(ids, skip_special_tokens=True) for ids in batch["source_ids"]]
                dec = [tokenizer.decode(ids, skip_special_tokens=True) for ids in outs]
                target = [tokenizer.decode(ids, skip_special_tokens=True) for ids in batch["target_ids"]]

            inputs.extend(input_)
            outputs.extend(dec)
            targets.extend(target)

    # decode_txt = "constrained" if is_constrained else "greedy"
    # with open(os.path.join(args.inference_dir, f"{name}_{decode_txt}_output.txt"), "w") as f:
    #     for i, o in enumerate(outputs):
    #         f.write(f"{inputs[i]} ===> {o}\n")

    # return inputs, outputs, targets
    return inputs, outputs


In [18]:
input_infer, output_infer = infer_new(
        args, dataset, model, tokenizer, 
        # name=f"thai-pred",
        # is_constrained=True, 
        is_constrained=False, 
        constrained_vocab=prepare_constrained_tokens(tokenizer, args.task, args.paradigm),
    )


for x,y in zip(input_infer, output_infer):
    print(f'input text:{x}')
    print(f'predict(gene) text:{y}')

Generating:   0%|          | 0/1 [00:00<?, ?it/s]

Input length of input_ids is 128, but ``max_length`` is set to 128. This can lead to unexpected behavior. You should consider increasing ``config.max_length`` or ``max_length``.


AttributeError: 'BaseModelOutputWithPoolingAndCrossAttentions' object has no attribute 'logits'